In [9]:
import pandas as pd
import regex as re
import pygsheets
from datetime import date, timedelta, datetime
import numpy as np
import geopandas as gpd

In [10]:
def get_revcircle_damages_dict(ROW):    
    DETAIL = ROW['Details']
    REV_CIRCLE_DAMAGES_DICT = dict()
    
    count = 0
    rev_circles_identified = []
    rev_circles_identifed_at = []
    for REV_CIRCLE in REVENUE_CIRCLES:
        if REV_CIRCLE in DETAIL:
            rev_circles_identified.append(REV_CIRCLE)
            rev_circles_identifed_at.append(DETAIL.index(REV_CIRCLE))
            count = count+1
    REV_CIRCLES_IDENTIFIED_DF = pd.DataFrame([rev_circles_identified,rev_circles_identifed_at]).T
    REV_CIRCLES_IDENTIFIED_DF.columns = ['RevenueCircle','StringIndex']
    REV_CIRCLES_IDENTIFIED_DF = REV_CIRCLES_IDENTIFIED_DF.sort_values(by='StringIndex').reset_index(drop=True)
    
    for i,val in enumerate(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
        StringStartIndex = val
        if i+1 == len(REV_CIRCLES_IDENTIFIED_DF.StringIndex):
            StringEndIndex = None
        else:
            StringEndIndex = REV_CIRCLES_IDENTIFIED_DF.StringIndex[i+1]
        
        SUB_DETAIL = DETAIL[StringStartIndex:StringEndIndex]
        
        if other_damages == True:
            NO_DAMAGES = len(re.findall('\|', SUB_DETAIL))
            print(NO_DAMAGES)
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES-1
            continue 
            
        NO_DAMAGES_DateAlgo = len(re.findall('2023',SUB_DETAIL)) + len(re.findall(r'\d+/\d+/\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+-\d+-\d{2}\s+', SUB_DETAIL)) + len(re.findall(r'\d+\.\d+\.\d{2}\s+', SUB_DETAIL))
        # Number of 2022 in the DETAIL +  Number of dd/mm/22 in the DETAIL +  Number of dd-mm-22 in the DETAIL + Number of dd.mm.22 in the DETAIL
        NO_DAMAGES_LongAlgo = len(re.findall('Long -',SUB_DETAIL)) 
    
        if float(ROW['Number of Damages (Long)'])==float(ROW['Number']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        elif float(ROW['Number of Damages (Long)'])>float(ROW['Number of Damages (Date)']):
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_LongAlgo
        else:
            REV_CIRCLE_DAMAGES_DICT[REV_CIRCLES_IDENTIFIED_DF.RevenueCircle[i].split('-')[0]] = NO_DAMAGES_DateAlgo
    
    
    return REV_CIRCLE_DAMAGES_DICT    

In [11]:
ASSAM_REVENUE_CIRCLES = pd.read_csv('Data/Assam_Maps/ASSAM_REVENUE_CIRCLES_FRIMS_NAMES.csv')
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-I\)','', regex=True)
ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.replace(r'\(Pt-II\)','', regex=True)

ASSAM_REVENUE_CIRCLES.revenue_ci = ASSAM_REVENUE_CIRCLES.revenue_ci.str.strip()

# Table of Contents:
* [Roads damaged per revenue circle](#roadsdamaged)
* [Embankment damages per revenue circle](#embankmentdamages)
* [Embankments affected per revenue circle](#embankmentaffected)
* [Bridges damaged per revenue circle](#bridgesdamaged)

## EMBANKMENT AFFECTED <a class="anchor" id="embankmentaffected"></a>

In [126]:
embankments_affected_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv')


embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Baghbar','Baghbor',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dotma','Dotoma',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Palashbari','Palasbari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
embankments_affected_df['Details'] = embankments_affected_df.Details.str.replace('Naduar','Na-Duar',regex=True)


embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
embankments_affected_df['District'] = embankments_affected_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


embankments_affected_df.tail()

,Date,District,Number,Details
31,2023-06-24,Nalbari,10,Nalbari - Pagaldia embankment | Bhadra | As pe...
32,2023-06-24,Kamrup,3,Hajo - Puthimari embankment | Hadal manakuchi ...
33,2023-06-24,Golaghat,1,Bokakhat - Un embankment portion of Alengmari ...
34,2023-06-24,Biswanath,1,Gohpur - Mild bank erosion is observed at R/Ba...
35,2023-06-25,Nalbari,7,Nalbari - Pagaldia embankment RB | Moiradanga ...


In [127]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_affected_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_affected_df[embankments_affected_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Tamulpur
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
Golaghat
['Bokakhat -', 'Dergaon -', 'Golaghat -', 'Khumtai -', 'Morangi -', 'Sarupathar -']
----
Goalpara
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
Karimganj
['Badarpur -', 'Karimganj -', 'Nilambazar -', 'Patherkandi -', 'RK Nagar -']
----
Lakhimpur
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
Nagaon
['Dhing -', 'Kaliabor -', 'Kampur -', 'Nagaon -', 'Raha -', 'Rupahi -', 'Samaguri -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Bajali
['Bajali -', 'Jalah -', 'Sarupeta -']
-

/tmp/ipykernel_5402/797364889.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [128]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [129]:
embankment_affected_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_affected_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_affected_revcircle_df.loc[i,'District'] = row['District']
        embankment_affected_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_affected_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_affected_revcircle_df.Date = pd.to_datetime(embankment_affected_revcircle_df.Date)

In [130]:
embankment_affected_revcircle_df.RevenueCircle =embankment_affected_revcircle_df.RevenueCircle.str.strip()
embankment_affected_revcircle_df.loc[embankment_affected_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [131]:
embankment_affected_revcircle_df = pd.merge(embankment_affected_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [132]:
embankment_affected_revcircle_df = embankment_affected_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [133]:
embankment_affected_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_EMBANKMENTS_AFFECTED_MASTER_2023.csv', index=False)

## EMBANKMENT BREACHES <a class="anchor" id="embankmentbreaches"></a>

In [100]:
embankments_breached_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')

embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Baghbar','Baghbor',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Dotma','Dotoma',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Palashbari','Palasbari',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
embankments_breached_df['Details'] = embankments_breached_df.Details.str.replace('Naduar','Na-Duar',regex=True)


embankments_breached_df['District'] = embankments_breached_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
embankments_breached_df['District'] = embankments_breached_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


embankments_breached_df.tail()

,Date,District,Number,Details
17,2023-06-22,Biswanath,1,Gohpur - Chatrang river R/Bank embankment brea...
18,2023-06-22,Nalbari,1,Nalbari - Pagaldia river at Nalbari sonkuriha ...
19,2023-06-23,Biswanath,1,Gohpur - Chatrang P&R D Bund | At L/Bank or Ri...
20,2023-06-24,Darrang,4,Mangaldoi - CM Dutta Bund FIS | At village Mol...
21,2023-06-25,Darrang,3,Mangaldoi - CM Dutta | At village Mollapara | ...


In [101]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in embankments_breached_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = embankments_breached_df[embankments_breached_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Lakhimpur
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Goalpara
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
Udalguri
['Dalgaon -', 'Dhekiajuli -', 'Harisinga -', 'Kalaigaon -', 'Khoirabari -', 'Mangaldoi -', 'Mazbat -', 'Pathorighat -', 'Udalguri -']
----
Sonitpur
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
Nagaon
['Dhing -', 'Kaliabor -', 'Kampur -', 'Nagaon -', 'Raha -', 'Rupahi -', 'Samaguri -']
----
Baksa
['Baganpara -', 'Barama -', 'Barnagar -', 'Baska -', 'Ghograpar -', 'Jalah -', 'Sarupeta -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Bajali
['Bajali -', 'Jalah -', 'Sarupeta -']
----


/tmp/ipykernel_5402/1701030496.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [102]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [103]:
embankment_breaches_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        embankment_breaches_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        embankment_breaches_revcircle_df.loc[i,'District'] = row['District']
        embankment_breaches_revcircle_df.loc[i,'Number of damages'] = no_damages
        embankment_breaches_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
embankment_breaches_revcircle_df.Date = pd.to_datetime(embankment_breaches_revcircle_df.Date)

In [104]:
embankment_breaches_revcircle_df.RevenueCircle =embankment_breaches_revcircle_df.RevenueCircle.str.strip()
embankment_breaches_revcircle_df.loc[embankment_breaches_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [105]:
embankment_breaches_revcircle_df = pd.merge(embankment_breaches_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [106]:
embankment_breaches_revcircle_df = embankment_breaches_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [107]:
embankment_breaches_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv', index=False)

## BRIDGES DAMAGED <a class="anchor" id="bridgesdamaged"></a>

In [63]:
bridges_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv')


bridges_df['Details'] = bridges_df.Details.str.replace('Baghbar','Baghbor',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dotma','Dotoma',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Palashbari','Palasbari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
bridges_df['Details'] = bridges_df.Details.str.replace('Naduar','Na-Duar',regex=True)


bridges_df['District'] = bridges_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
bridges_df['District'] = bridges_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


bridges_df.tail()

,Date,District,Number,Details
19,2023-06-23,Barpeta,3,Baghbor - 1. RCC BR No. 7/1 (Barpeta Baghbor r...
20,2023-06-23,Bajali,2,Sarupeta - RCC Br. No. 2/1 | Sarupeta Gaon | R...
21,2023-06-24,Dhubri,1,Dhubri - APPROACHES OF SPT BR. NO. 3/1 AND ROA...
22,2023-06-25,Bongaigaon,2,Srijangram - Golapara to Kirtanpara road | Gol...
23,2023-06-25,Kokrajhar,1,Dotoma - Rabhapara ward no.4 to Ward no5 conne...


In [64]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in bridges_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = bridges_df[bridges_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2022', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Sonitpur
['Chariduar -', 'Dhekiajuli -', 'Na-Duar -', 'Tezpur -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Kokrajhar
['Bagribari -', 'Bhawraguri -', 'Chapar -', 'Dhubri -', 'Dotoma -', 'Golokganj -', 'Gossaigaon -', 'Kokrajhar -']
----
Barpeta
['Baghbor -', 'Barnagar -', 'Barpeta -', 'Chenga -', 'Kalgachia -', 'Sarthebari -']
----
Bongaigaon
['Boitamari -', 'Bongaigaon -', 'Manikpur -', 'Sidli -', 'Srijangram -']
----
Baksa
['Baganpara -', 'Barama -', 'Barnagar -', 'Baska -', 'Ghograpar -', 'Jalah -', 'Sarupeta -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Dhubri
['Agamoni -', 'Bagribari -', 'Bilasipara -', 'Chapar -', 'Dhubri -', 'Golakganj -', 'Gossaigaon -']
----
Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Bajali
['Bajali -', 'J

/tmp/ipykernel_5402/1577161483.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [65]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [66]:
bridges_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        bridges_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        bridges_revcircle_df.loc[i,'District'] = row['District']
        bridges_revcircle_df.loc[i,'Number of damages'] = no_damages
        bridges_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
bridges_revcircle_df.Date = pd.to_datetime(bridges_revcircle_df.Date)

In [67]:
bridges_revcircle_df.RevenueCircle =bridges_revcircle_df.RevenueCircle.str.strip()
bridges_revcircle_df.loc[bridges_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [68]:
bridges_revcircle_df = pd.merge(bridges_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [87]:
bridges_revcircle_df = bridges_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [88]:
bridges_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_BRIDGES_DAMAGED_MASTER_2023.csv', index=False)

## ROADS DAMAGED <a class="anchor" id="roadsdamaged"></a>

In [89]:
roads_damaged_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_ROADS_DAMAGED_MASTER_2023.csv')

roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Baghbar','Baghbor',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dotma','Dotoma',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Palashbari','Palasbari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('North Ghy','North\nGuwahati',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Khairabari','Khoirabari',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Dangtol','Bongaigaon',regex=True)
roads_damaged_df['Details'] = roads_damaged_df.Details.str.replace('Naduar','Na-Duar',regex=True)


roads_damaged_df['District'] = roads_damaged_df.District.str.replace('Dima-Hasao','Dima Hasao',regex=True)
roads_damaged_df['District'] = roads_damaged_df.District.str.replace('Sivasagar','Sivsagar',regex=True)


roads_damaged_df.head()

,Date,District,Number,Details
0,2023-06-11,Biswanath,1,Gohpur - Singarajan to Sutardoloni Kutcha Road...
1,2023-06-12,Biswanath,6,Gohpur - Road from Borigaon to Deonabori. (A c...
2,2023-06-12,Darrang,1,Mangaldoi - Approach road Noanadi Kachia Bund ...
3,2023-06-13,Lakhimpur,1,Narayanpur - Balikuchi Bishnupur Road at Ch 35...
4,2023-06-14,Dhemaji,2,Dhemaji - PWD Deptt. Road from Jiadhal charial...


In [90]:
roads_damaged_df = roads_damaged_df.sort_values(by='Date')

In [91]:
global REVENUE_CIRCLES
global other_damages
other_damages = False

FILTERED_DAMAGES_dfs = []

for DISTRICT in roads_damaged_df.District.unique():
    print(DISTRICT)
    FILTERED_DAMAGES_df = roads_damaged_df[roads_damaged_df.District == DISTRICT]
    FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.sort_values(by=['Detail_Length'])
    
    FILTERED_DAMAGES_df = FILTERED_DAMAGES_df.drop_duplicates(['District','Number','Date'],keep='last')
    
    FILTERED_DAMAGES_df['Number of Damages (Date)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('2023', x))+len(re.findall(r'\d+/\d+/\d{2}\s+',x))+len(re.findall(r'\d+-\d+-\d{2}\s+',x))+len(re.findall(r'\d+\.\d+\.\d{2}\s+', x)))
    FILTERED_DAMAGES_df['Number of Damages (Long)'] = FILTERED_DAMAGES_df.Details.apply(lambda x: 
                                     len(re.findall('Long -', x)))
        
    
    REVENUE_CIRCLES = [REV_CIRCLE.strip()+' -' for REV_CIRCLE in list(ASSAM_REVENUE_CIRCLES[ASSAM_REVENUE_CIRCLES.district_35==DISTRICT.upper()].revenue_ci)]
    print(REVENUE_CIRCLES)
    print('----')
    FILTERED_DAMAGES_df['NumberofDamages_RevenueCircle_level'] = FILTERED_DAMAGES_df.apply(get_revcircle_damages_dict, axis=1)
    FILTERED_DAMAGES_dfs.append(FILTERED_DAMAGES_df)

Biswanath
['Biswanath -', 'Gohpur -', 'Halem -']
----
Darrang
['Dalgaon -', 'Khoirabari -', 'Mangaldoi -', 'Patharighat -', 'Sipajhar -']
----
Lakhimpur
['Bihpuria -', 'Dhakuakhana -', 'Kadam -', 'Narayanpur -', 'North Lakhimpur -', 'Nowboicha -', 'Subansiri -']
----
Dhemaji
['Dhakuakhana -', 'Dhemaji -', 'Gogamukh -', 'Jonai -', 'Sissiborgaon -', 'Subansiri -']
----
Dima Hasao
['Haflong -', 'Mahur -', 'Maibong -', 'Umrangso -']
----
Nalbari
['Baganpara -', 'Banekuchi -', 'Barbhag -', 'Barkhetri -', 'Ghograpar -', 'Nalbari -', 'Pachim\nNalbari -', 'Tihu -']
----
Goalpara
['Balijana -', 'Dudhnoi -', 'Lakhipur -', 'Matia -', 'Rangjuli -']
----
Tamulpur
['Bajali -', 'Goreswar -', 'Pathorighat -', 'Rangia -', 'Tamulpur -']
----
Udalguri
['Dalgaon -', 'Dhekiajuli -', 'Harisinga -', 'Kalaigaon -', 'Khoirabari -', 'Mangaldoi -', 'Mazbat -', 'Pathorighat -', 'Udalguri -']
----
Kamrup
['Azara -', 'Boko -', 'Chamaria -', 'Chhaygaon -', 'Goroimari -', 'Hajo -', 'Jalah -', 'Kamalpur -', 'Kayan -',

/tmp/ipykernel_5402/993252146.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  FILTERED_DAMAGES_df['Detail_Length'] = FILTERED_DAMAGES_df['Details'].apply(lambda x: len(x))


In [92]:
MASTER_DAMAGES = pd.concat(FILTERED_DAMAGES_dfs).reset_index(drop=True)
MASTER_DAMAGES.District = MASTER_DAMAGES.District.str.upper()

In [93]:
road_damages_revcircle_df = pd.DataFrame(columns=['District','RevenueCircle','Number of damages','Date'])
i = 0
for idx, row in MASTER_DAMAGES.iterrows():
    damages_dict = row['NumberofDamages_RevenueCircle_level']
    for key in damages_dict.keys():
        rev_circle = key
        no_damages = damages_dict[key]
        
        road_damages_revcircle_df.loc[i,'RevenueCircle'] = rev_circle
        road_damages_revcircle_df.loc[i,'District'] = row['District']
        road_damages_revcircle_df.loc[i,'Number of damages'] = no_damages
        road_damages_revcircle_df.loc[i,'Date'] = row['Date']
        i=i+1
road_damages_revcircle_df.Date = pd.to_datetime(road_damages_revcircle_df.Date)

In [94]:
road_damages_revcircle_df.RevenueCircle =road_damages_revcircle_df.RevenueCircle.str.strip()
road_damages_revcircle_df.loc[road_damages_revcircle_df.RevenueCircle == 'Na', 'RevenueCircle'] = 'Na-Duar'

In [95]:
road_damages_revcircle_df = pd.merge(road_damages_revcircle_df,
                                ASSAM_REVENUE_CIRCLES[['district_35', 'revenue_ci','object_id']],
                                left_on=['District', 'RevenueCircle'],
                                right_on=['district_35', 'revenue_ci'],
                                how='left')

In [96]:
road_damages_revcircle_df = road_damages_revcircle_df[['Date','object_id','revenue_ci','District','Number of damages']]

In [97]:
road_damages_revcircle_df.to_csv('Data_2023/Cleaned Data/RC_FRIMS_ROADS_DAMAGED_MASTER_2023.csv', index=False)

In [194]:
embankments_breached_df = pd.read_csv('Data_2023/Cleaned Data/FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')

In [203]:
longitudes_list = []
latitudes_list = []

for idx, row in embankments_breached_df.iterrows():
    details = row['Details']
    long_lats = re.findall('\d{2}\.\d{6}', details)
    longitudes = list(filter(lambda x: float(x) > 40, long_lats))
    longitudes_list.append(longitudes)
    
    latitudes = list(filter(lambda x: float(x) < 40, long_lats))
    latitudes_list.append(latitudes)

In [208]:
longitudes_list = flatten_list(longitudes_list)
latitudes_list = flatten_list(latitudes_list)

In [212]:
breaches_lat_lon_df = pd.DataFrame([longitudes_list,latitudes_list]).T
breaches_lat_lon_df.columns = ['Longitude', 'Latitude']
breaches_lat_lon_df.to_csv('Data_2023/Cleaned Data/LAT_LONS_FRIMS_EMBANKMENTS_BREACHED_MASTER_2023.csv')

In [206]:
def flatten_list(nested_list):
    flattened_list = []
    for item in nested_list:
        if isinstance(item, list):
            flattened_list.extend(flatten_list(item))
        else:
            flattened_list.append(item)
    return flattened_list